# Data Exploration and Preparation

### Importing Important Libraries and Modules

In [1]:
from datasets import load_dataset, DatasetDict, Dataset
import spacy
import re
import os
import pandas as pd
import streamlit as st
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import json
nlp = spacy.load("en_core_web_sm")

C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## Loading the PubMed dataset for summarization

In [3]:

dataSet = load_dataset("ccdv/pubmed-summarization", trust_remote_code=True)



Using the latest cached version of the module from C:\Users\Crown Tech\.cache\huggingface\modules\datasets_modules\datasets\ccdv--pubmed-summarization\f765ec606c790e8c5694b226814a13f1974ba4ea98280989edaffb152ded5e2b (last modified on Sat Jun 22 13:13:24 2024) since it couldn't be found locally at ccdv/pubmed-summarization, or remotely on the Hugging Face Hub.


## Exploring the dataset

In [4]:
print(dataSet)

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 119924
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})


In [9]:
anyDataInDataSet = dataSet['validation'][55]
print("Artical Part is..........."   )
print(anyDataInDataSet['article']  )
print("")
print("")
print("-----------------------------------------------------------------------------------------------------------------------------------------------")
print("")
print("")

print("Abstract Part is..........."   )
print(anyDataInDataSet['abstract']  )




Artical Part is...........
diabetes mellitus is a growing public health concern , presently affecting 25.8 million or 8.3% of the american population   and nearly 387 million people worldwide . 
 while the availability of novel drugs , techniques , and surgical intervention has improved the survival rate of individuals with diabetes , the prevalence of diabetes is still rising in the united states , with the number of people with diabetes projected to double by 2025 . 
 t2d is a result of chronic insulin resistance and loss of -cell mass and function . 
 both in experimental animals and people , obesity is a leading pathogenic factor for developing insulin resistance , which is always associated with the impairment in energy metabolism , causing increased intracellular fat content in skeletal muscle , liver , fat , and pancreatic islets . 
 constant insulin resistance will progress to t2d when -cells are unable to secrete adequate amount of insulin to compensate for decreased insulin s

### Contents And Structure Of the Dataset

- The dataset **ccdv/pubmed-summarization** has three splits **train**, **validation** and **test** Dataset.
- DataSet has two features **['article', 'abstract']**
- Train Dataset is higly content in data.
- Data is already in smallcase letters.
- Data contains special Chars.
- Data contains URLs.

## Preprocessing and Data Cleaning (Only Run One Time As the Initial Cleaned Data is Stored On Pc)

In [5]:
# not doing tokenization here as we have a huge dataset we can do tokenization on a small set 

def initialCleaningOfData(text):
    # remove extra spaces, newlines, and specified special characters
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'\n', ' ', text)   # remove newlines
    text = re.sub(r'[^\w\s]', '', text)  # remove all non-word and non-space characters
    text = re.sub(r'http\S+|www.\S+', '', text)  # remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # remove email addresses
    text = re.sub(r'\d+', '', text)  # remove numbers
    return text.strip()

def prepareAndCleanMyDataset(dataSet):
    dataSet['article'] = initialCleaningOfData(dataSet['article'])
    dataSet['abstract'] = initialCleaningOfData(dataSet['abstract'])
    return dataSet

#beautiful Func save me alot of time :)
dataSet = dataSet.map(prepareAndCleanMyDataset, batched=False)

output_file = 'cleaned_dataset.json'

data_list = []
for split_name in dataSet.keys():
    for example in dataSet[split_name]:
        data_list.append({
            'split': split_name,
            'article': example['article'],
            'abstract': example['abstract']
        })

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"Cleaned dataset saved to {output_file}")



Cleaned dataset saved to cleaned_dataset.json


### Run This Block Once you Run Above one Every Time

In [7]:
#saving it locally to save time and ease of loading
input_file = 'D:/cleaned_dataset.json' #D:/cleaned_dataset.json
with open(input_file, 'r', encoding='utf-8') as f:
    dataList = json.load(f)

In [3]:
print(dataList[119930])

{'split': 'validation', 'article': 'most adults with autoimmune diabetes non  insulin  requiring at diagnosis become so within  years       the optimal treatment for this second largest group of patients with diabetes is still unknown     adult patients with autoimmune diabetes usually have larger remaining cell mass at diagnosis and many develop cell destruction more slowly  latent autoimmune diabetes in adults  lada  is therefore a suitable group for evaluating new therapies in autoimmune diabetes and may also serve as a model for intervention in classical type  diabetes         the incidence of autoimmune diabetes is about equal in almost all age groups       abrupt onset  often with ketoacidosis  is most frequent during childhood  a more modest onset is more frequent in adolescents and younger adults  and among adults and elders a slowly progressive onset  termed lada  is frequent           classical type  diabetes and lada patients often have normal c  peptide levels at diagnosis 

## Tokenization And Lemmatization (of Limited Part Of Data Set)

In [4]:

def removeStopwordsAndLemmatize(text):
    
    doc = nlp(text)
    
    lemmatizedTokens = []
    
    for token in doc:
        
        if not token.is_stop:
            
            lemmatizedTokens.append(token.lemma_)
    
    return ' '.join(lemmatizedTokens)

def prepareAndClean(example):
    example['article'] = removeStopwordsAndLemmatize(example['article'])
    example['abstract'] = removeStopwordsAndLemmatize(example['abstract'])
    return example


trainData, validationData, testData = [], [], []
for example in dataList:
    if example['split'] == 'train' and len(trainData) < 500:
        trainData.append(example)
    elif example['split'] == 'validation' and len(validationData) < 500:
        validationData.append(example)
    elif example['split'] == 'test' and len(testData) < 500:
        testData.append(example)



trainData = [prepareAndClean(example) for example in trainData]
validationData = [prepareAndClean(example) for example in validationData]
testData = [prepareAndClean(example) for example in testData]






In [4]:
print(validationData[4])


{'split': 'validation', 'article': 'need magnetic resonance image   mri   patient implanted pacemaker implantable cardioverter   defibrillator   icd   grow clinical issue   estimate   active cardiac device recipient indicate mri   currently   vast majority device contraindicate use mri   european heart rhythm association survey   publish recently non   mri   certify icd    t field strength    totally subcutaneous icd   s   icd   system   implantable defibrillator lead touch heart   recently demonstrate safe effective defibrillator option patient risk sudden cardiac death   provide shock therapy post   shock pace therapy   long   term bradycardia pace   show alternative standard transvenous icd   compatibility mri remain unclear   type clinical mri system currently use superconductive magnet create static magnetic field strength   typically     t use mri pacemaker icd consider contraindication potential hazard   include heating electrode reside heart   damage myocardium   elevation pace

## Training And Testing Of Model

In [8]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(trainData)),
    'validation': Dataset.from_pandas(pd.DataFrame(validationData)),
    'test': Dataset.from_pandas(pd.DataFrame(testData))
})

NameError: name 'trainData' is not defined

## T5 Model for Summarization

In [7]:
modelName = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(modelName)
model = T5ForConditionalGeneration.from_pretrained(modelName)

C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\huggingface_hub-0.23.4-py3.8.egg\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Reasons For Selection Of Model
- Customization: T5 can be fine-tuned to specific domains or datasets, improving its performance on the specific summarization task you are interested in.
- Performance: T5 has shown strong performance on multiple benchmarks, including summarization tasks, making it a reliable choice for generating concise and accurate summaries.
- Pre-trained Model: T5 is pre-trained on a large corpus of text, which gives it a strong understanding of language. This pre-training helps the model perform well on various tasks, including summarization, after fine-tuning on specific datasets.

In [8]:
def preprocessFunction(examples):
    inputs = [f"summarize: {doc}" for doc in examples['article']]
    modelInputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['abstract'], max_length=150, padding='max_length', truncation=True)

    modelInputs['labels'] = labels['input_ids']
    return modelInputs

tokenizedDataset = dataset.map(preprocessFunction, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,  # Adjusted learning rate
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    lr_scheduler_type='linear',  # Example of using a linear scheduler
    logging_dir='./logs',
    logging_steps=10,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizedDataset['train'],
    eval_dataset=tokenizedDataset['validation'],
    tokenizer=tokenizer,
    # Optional: Add additional callbacks, logging, etc.
)


C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\accelerate\accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [10]:

# Fine-tune the model
trainer.train()

# Define the directory where the model will be saved
# Use the home directory for the user
home_directory = os.path.expanduser('~')
save_directory = os.path.join(home_directory, 'fine_tuned_t5_model')

# Create the directory if it does not exist
os.makedirs(save_directory, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Model fine-tuned and saved.")


Epoch,Training Loss,Validation Loss
1,3.968800,3.512763
2,3.658000,3.439108
3,3.931300,3.420180


Model fine-tuned and saved.


## Performane Of Our Model On New Data

In [9]:
import re
import spacy
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load SpaCy model for text preprocessing
nlp = spacy.load("en_core_web_sm")

# Function to clean and prepare new input text
def cleanText(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\n', ' ', text)   # Remove newlines
    text = re.sub(r'[^\w\s]', '', text)  # Remove all non-word and non-space characters
    text = re.sub(r'http\S+|www.\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # Remove email addresses
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.lower()  # Convert text to lowercase
    return text.strip()

def removeStopwordsAndLemmatize(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc if not token.is_stop])

def prepareAndClean(text):
    text = cleanText(text)
    text = removeStopwordsAndLemmatize(text)
    return text

# Load the fine-tuned model and tokenizer
home_directory = os.path.expanduser('~')
save_directory = os.path.join(home_directory, 'fine_tuned_t5_model')

model = T5ForConditionalGeneration.from_pretrained(save_directory)
tokenizer = T5Tokenizer.from_pretrained(save_directory)

# Function to generate a summary for a new PubMed article
def generateSummary(article_text):
    cleaned_text = prepareAndClean(article_text)
    input_text = f"summarize: {cleaned_text}"
    
    inputs = tokenizer(input_text, max_length=512, padding='max_length', truncation=True, return_tensors="pt")
    summary_ids = model.generate(inputs.input_ids, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage
new_article_text = """
their radiological features suggest malignant neoplasms  whereas they are not ct and mr imaging are the most used tools in their evaluation inflammatory myofibroblastic tumours  imts  constitute a rare group of neoplasms composed of a mixture of spindle  shaped myofibroblasts or fibroblasts and a variable amount of inflammatory cells  eosinophils  plasma cells and lymphocytes   many different terms have been used to refer to these tumours  plasma cell granuloma  inflammatory myofibrohistiocytic proliferation  fibroxanthoma  xanthogranuloma  the most frequently affected organs are lung and orbit  but they have been described in nearly every organ  different aetiologies have been proposed for imt  being different chronic infections  autoimmune diseases and trauma the most accepted  specific inflammatory diseases  such as igg disease  have also been recently associated       an aggressive behaviour with metastases has been described         clinical presentation of imts depends on the organ in which they arise  but they frequently associate general inflammatory symptoms as fever or malaise  radiological appearance of imts is unspecific and they are often misdiagnosed as malignant neoplasms  many of them are incidentally discovered when an imaging technique  computed tomography  ct   ultrasonography  us  or magnetic resonance imaging  mri   is performed for any other reason  their most common radiological presentation is as solid  irregular  well  defined masses  immunohistochemical studies of t and b  cell subpopulations  cd  cd  s and c  kit may be helpful in distinguishing imts from other soft  tissue neoplasms       at the molecular level  approximately half of imts contain a clonal cytogenetic aberration that activates the anaplastic lymphoma kinase  alk   positive immunohistochemical staining of alk is in approximately   of imts  depending on the anatomical sites where they arise     furthermore  alk expression could be a prognostic factor of aggressiveness for imt  according to an update based on the new world health organisation  who  classification  imts are considered as neoplasms which may recur or metastasise in as many as   of cases  the objective of this pictorial review is to present the most important characteristics of imts arising in different locations of the body with their histological correlation   these tumours can appear as hypoechoic or hyperechoic masses with ill  defined or well  circumscribed edges and a variable doppler appearance with increased vascularity  us is usually the first imaging technique when imts appear in specific locations  such as testicles or neck  in other locations on contrast  enhanced ct  imts can appear as homogeneous or heterogeneous lesions  with variable enhancement on delayed acquisitions due to the presence of fibrosis  these findings are also present on gadolinium contrast  enhanced mri  on tweighted and tweighted sequences  imts usually show low signal intensity reflecting also the presence of fibrotic tissue  table table symptoms  radiological findings and differential diagnosis of the main imts of the bodysymptomsimaging findingsdifferential diagnosisorbitpain  redness  oedema  proptosis  ptosis  oculomotor deficits  diplopia  swelling  mass  effectsolid and heterogeneous enhancing  associated retrobulbar fat or oedemagranulomatous diseases  primary infection  sarcoid  sjgren disease  primary and secondary tumours of the orbit  lymphoma and connective tissue diseaseslungcough  chest pain  dyspnea  haemoptysis  fever  malaise and weight losssolitary  well  circumscribed  peripheral  predominance for the lower lobes  unusual calcifications  low intensity on tweighted imagesmalignant lung masses  primary bronchogenic carcinomascrotumlump in the scrotumsolid and heterogeneous masses with internal vascularity on us  small calcifications can be associatedscrotum primary neoplasmhepaticunspecific  depending on the sizeheterogeneous or peripheral enhancement during the arterial phase  or homogeneous enhancement during the arterial phase and washout during the delayed phase on ct  t hypointense and t hyperintense with heterogeneous enhancementhccsplenicleft upper quadrant or epigastric painlow  density pattern  low  intensity lesion in the early phase after gadolinium injection and high  intensity in the delayed phasesplenic haemangioma and other primary splenic neoplasms as lymphoma or splenic angiosarcomagrastrointestinal tract and mesenteryabdominal pain  intestinal obstruction  dysphagia  anaemiahypodense  heterogeneous  well  delimitedgist  inflammatory fibroid polyp  smooth muscle neoplasm  peripheral nerve sheath tumour  solitary fibrous tumour  fibromatosis  the follicular dendritic cell sarcoma  lymphoma and adenocarcinomamusculoskeletalpain  oedemanon  homogeneous  solidmalignant lesions  rabdomyosarcoma symptoms  radiological findings and differential diagnosis of the main imts of the body imts have been reported in various sites in the head and neck such as bucal space  maxillary sinus  nasal cavity  parotid gland  nasopharynx and larynx  fig  pathology  imt in the left supraglottic space  arrow  with paraglottic space involvement indicated by replacement of the paraglottic fat with soft tissue  a axial reformation of contrast  enhanced neck ct  a solid  well  defined  normal tissue was present in the peripheral zone    pathology  imt in the left supraglottic space  arrow  with paraglottic space involvement indicated by replacement of the paraglottic fat with soft tissue  a axial reformation of contrast  enhanced neck ct  a solid  well  defined  little  enhancing nodule in the supraglottic larynx was observed  yellow arrow   b microscopic study obtained after surgical removal of the lesion  the sample showed proliferation of fusiform cells mixed with macrophages and giant multinucleated cells  normal tissue was present in the peripheral zone    however  the orbit is the most common location of imts  they account for approximately   of orbital masses  being the third most common cause of orbital masses  their aetiology is unknown  but they have been associated with inmunological disorders  different infections and history of trauma  recently  they have been strongly associated with the iggrelated sclerosing systemic inflammatory disease and with chronic infection by epstein  barr virus  the clinical presentation and symptoms may vary with the location  extension and aggressiveness of the tumour  patients with imts of the orbit can present local symptoms as pain  redness  oedema  proptosis  ptosis  oculomotor deficits  diplopia  lid swelling or mass  effect and systemic symptoms due to the general inflammatory disorders they are frequently associated with  mri is superior to ct in the evaluation of inflammation of the nerves and muscles  they usually appear as solid and heterogeneous enhancing masses  which can affect retro bulbar fat  cause bone destruction  and present intracranial extension  when they show low intensity on tweighted images prominent  fibrotic composition is present  fig  a  b axial and coronal mr reformations of the same patient on tweighted sequences showed a well  delimited  hyperintense intraorbital mass  in the intraconal compartment  in the medial aspect  c  d tweighted images demonstrated a hypointense  homogeneous  well  delimited mass with peripheral enhancement after gadolinium injection  f microscopic sample obtained after surgery demonstrated an epithelial tumour composed of fusiform cells mixed with an extensive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  in the peripheral of the sample a  b axial and coronal mr reformations of the same patient on tweighted sequences showed a well  delimited  hyperintense intraorbital mass  in the intraconal compartment  in the medial aspect  c  d tweighted images demonstrated a hypointense  homogeneous  well  delimited mass with peripheral enhancement after gadolinium injection  f microscopic sample obtained after surgery demonstrated an epithelial tumour composed of fusiform cells mixed with an extensive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  in the peripheral of the sample striated muscle cells were also observed pathologically  imts of the head and neck consist of many fibroblastic cells  fibroblastic cells and inflammatory cells  which include plasma cells and eosinophils  the differential diagnosis includes granulomatous diseases  primary infection  sarcoid  sjgren disease  primary and secondary tumours of the orbit  lymphoma and connective tissue diseases  the administration of corticosteroids an inmunosupresors usually leads to a decrease of size of the mass but also radiotherapy or surgery may be indicated  they account for around the   of benign pulmonary masses in children  representing only about   of all tumours of the lung in general population  as in other organs sometimes  imts are also related with antecedents of lung surgery and they can arise in surgical lung scars  common clinical characteristics include unspecific respiratory symptoms  such as cough  chest pain  dyspnea  haemoptysis and unspecific inflammatory symptoms as fever  malaise and weight loss  they usually appear as solitary  well  circumscribed peripheral lung masses  with predominance for the lower lobes  fig  on ct with intravenous contrast  they present a variable heterogeneous or homogeneous degree of enhancement pattern       it has been described an aggressive behaviour with invasion of the adjacent structures  on mri because of their similar radiological appearance to malignant lung masses  biopsy is necessary to obtain the diagnosisfig  a year  old man with a cough and haemoptysisa axial reformation of non  contrast ct with mediastinum window  two solid  ill  defined nodules in right lower lobe are observed  yellow arrows   b axial reformations with lung window of the same patient where the lung nodules are shown  white arrows   c partial lobectomy of the right lower lobe specimen  note the presence of the nodules     d microscopic study of the resected lung sample where fusiform cells with an associated inflammatory infiltrate of lymphocytes  plasmatic cells and histiocytes was found pathology  imt of the lung  a year  old man with a cough and haemoptysisa axial reformation of non  contrast ct with mediastinum window  two solid  ill  defined nodules in right lower lobe are observed  yellow arrows   b axial reformations with lung window of the same patient where the lung nodules are shown  white arrows   c partial lobectomy of the right lower lobe specimen  note the presence of the nodules     d microscopic study of the resected lung sample where fusiform cells with an associated inflammatory infiltrate of lymphocytes  plasmatic cells and histiocytes was found the treatment of choice of lung imts is surgery to exclude malignancy and to achieve the cure  although spontaneous regression may occur  local expansion may cause significant morbidity and occasional death  the prognosis after complete surgical resection is excellent  with a low rate of recurrence  in these cases  alternative treatments  radiotherapy  corticoids or chemotherapy  to surgery could be also used  only a few cases of imts in the spermatic cord have been reported  the most common presentation of epididymal  fig    and paratesticular imts is a lump in the scrotumfig  pathology  imt of the epididymus  a year  old man with a lump in the scrotum  a a hypoechoic a diffuse infiltrate composed of fibroblastic and fusiform cells mixed with inflammatory cells  mainly lymphocytes and macrophages  was observed pathology  imt of the epididymus  a year  old man with a lump in the scrotum  a a hypoechoic  well  delimited epididymal lesion in the right scrotum was detected  a diffuse infiltrate composed of fibroblastic and fusiform cells mixed with inflammatory cells  mainly lymphocytes and macrophages  was observed in the evaluation of scrotal masses  us is the most accurate imaging technique  helping to distinguish intratesticular from extratesticular lesions and solid from cystic lesions  on ct  they appear as well  defined  hypodense masses with little enhancement after intravenous contrast injection  in some cases  a year  old man with no relevant medical history came to our hospital with a lump in the scrotum  a us showed an isoechoic  solid  paratesticular nodule with fine calcifications  yellow arrow   b axial reformation of contrast  enhanced ct on portal phase demonstrated a well  delimited  heterogeneous  little  enhancing mass in the right scrotum  d microscopic study of the lesion demonstrated a well  delimited lesion constituted of fibroblasts  lymphocytes and plasmatic cells with associated lymphoid follicle  a year  old man with no relevant medical history came to our hospital with a lump in the scrotum  a us showed an isoechoic  solid  paratesticular nodule with fine calcifications  yellow arrow   b axial reformation of contrast  enhanced ct on portal phase demonstrated a well  delimited  heterogeneous  little  enhancing mass in the right scrotum  d microscopic study of the lesion demonstrated a well  delimited lesion constituted of fibroblasts  lymphocytes and plasmatic cells with associated lymphoid follicle  there was no evidence of adjacent testicle invasion surgical resection without orchiectomy is the ideal treatment  but in some cases the testicle must be removed due to the size of the mass or when the mass is attached to the testis  imts of the penis are extremely rare  to our knowledge  no imts at this location have been reported  on ct  this tumour appeared as a well  defined  little  enhancing and non  aggressive mass surrounding the penile urethra and the corpora cavernosa with no evidence of invasion of themfig  a year  old man with a mass in the penis came to our hospital  he related more than  years of slow growth of the mass with no suspicious associated symptoms  c  d axial and coronal reformations of enhanced  ct after the injection of intravenous contrast on portal phase  macroscopic view showed an heterogeneous aspect due to the mixture of tissues  solid tumour  lipoid component and areas of haemorrhage   f microscopic study confirmed a mesenchymal tumour composed of fusiform cells  some of them multinucleated  with round or elongate nucleus  a year  old man with a mass in the penis came to our hospital  he related more than  years of slow growth of the mass with no suspicious associated symptoms  c  d axial and coronal reformations of enhanced  ct after the injection of intravenous contrast on portal phase  macroscopic view showed an heterogeneous aspect due to the mixture of tissues  solid tumour  lipoid component and areas of haemorrhage   f microscopic study confirmed a mesenchymal tumour composed of fusiform cells  some of them multinucleated  with round or elongate nucleus  no mitotic figures were observed surgical treatment was performed due to the size and non  conclusive radiological features of the tumour  hepatic imts constitute an extremely rare group of hepatic tumours  accounting only for the   of all the hepatic lesions  they have been associated with autoimmune diseases  immunological disorders and viral infections  such as igg disease  sclerosing cholangitis  crohns disease  hepatic viral infections and epstein  barr virus  symptoms of hepatic imts depend on their size  on contrast  enhanced ct  they usually show heterogeneous or peripheral enhancement during the arterial phase  despite this  some of them present homogeneous enhancement during the arterial phase and washout during the delayed phase  which can lead to misdiagnosing them as hepatocarcinomas  hccs   fig     on mri  these lesions usually are t hypointense and t hyperintense with heterogeneous enhancement  fig    hepatobiliary contrast  gadoxetate disodium  primovist  bayer healthcare  leverkusen  germany  could be of help to differentiate hcc from imts of the liver  on the hepatobiliary phase  high signal intensity in the centre of the lesion has been described  suggesting a benign lesionfig  well  defined  hypointense lesion on tweighted images  a  which presented weak enhancement in the arterial phase  b  and wash  out on portal and equilibrium phases  c  d  located in hepatic left lobe  f the lesion demostrated an expansive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  a abdominal ultrasound where a slightly hypoechoic  solid  heterogeneous mass arising in the left hepatic lobe was observed  axial tweighted unenhanced  b  and contrast  enhanced mr images  c  d  showed an ill  defined with heterogeneous enhancement mass located in lateral hepatic segments  mesenchymal tumour composed of fusiform cells  lymphocytes and lipid  filled macrophages can be observed  normal hepatocytes with large and small fatty droplet change are also present pathology  imt of the liver  well  defined  hypointense lesion on tweighted images  a  which presented weak enhancement in the arterial phase  b  and wash  out on portal and equilibrium phases  c  d  located in hepatic left lobe  e macroscopic view of partial hepatectomy obtained after surgical excision of the lesion  a solid  well  defined and yellowing appearance mass is showed  f the lesion demostrated an expansive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  a abdominal ultrasound where a slightly hypoechoic  solid  heterogeneous mass arising in the left hepatic lobe was observed  axial tweighted unenhanced  b  and contrast  enhanced mr images  c  d  showed an ill  defined with heterogeneous enhancement mass located in lateral hepatic segments  mesenchymal tumour composed of fusiform cells  lymphocytes and lipid  filled macrophages can be observed  normal hepatocytes with large and small fatty droplet change are also present surgery is the most accepted treatment  if there is a biopsy  proven diagnosis of imts that excludes malignancy  medical treatment with no steroidal anti  inflammatory drugs in patients with peripheral hepatic imts could be prescribed  patients with spleen imts frequently associate unspecific symptoms as left upper quadrant or epigastric pain  splenomegaly is common and some patients can present fever  anaemia and signs of hypersplenism  different radiological imaging modalities have been proposed for their evaluation  on all of them  they are depicted as low  density masses in both non  enhanced and enhanced acquisitions  fig     on mri studies  they present low  intensity mass on both the t and tweighted images  their typical enhancement pattern after gadolinium injection is as a low  intensity lesion in the early phase  which changes into a high  intensity one in the delayed phase fig  pathology  imt of the spleen  a axial reformation of contrast  enhanced ct on portal phase  prominent cellularity composed of an irregular  weakly basophil set of mioepithelial  fusiform cells  numerous inflammatory cells  plasmatic  lymphocytes and few neutrophils and eosinophils  with isolated macrophages were also found  no atypia or mitotic figures were present pathology  imt of the spleen  a axial reformation of contrast  enhanced ct on portal phase  prominent cellularity composed of an irregular  weakly basophil set of mioepithelial  fusiform cells  numerous inflammatory cells  plasmatic  lymphocytes and few neutrophils and eosinophils  with isolated macrophages were also found  no atypia or mitotic figures were present the differential diagnosis includes splenic haemangioma and other primary splenic neoplasms such as lymphoma or splenic angiosarcoma  when a suspicious spleen mass is found  the treatment of choice is splenectomy due to the high risk of bleeding associated to splenic biopsy       they can arise anywhere  the stomach and small intestine being the most common locations       clinical symptoms may vary from none to abdominal pain  intestinal obstruction  dysphagia or anaemia  depending on the location and size of the mass  contrast  enhanced ct is the imaging technique  which give us more information about of imts of the gastrointestinal tract  they are usually found as hypodense  heterogeneous and well  delimited masses  figs  axial reformation of contrast  enhanced ct on arterial  a   portal  b  and  min after injection delayed  c  acquisitions  a well  defined  heterogeneous nodule with moderate enhancement on portal phase arising from the gastric fundus was found  yellow arrows   d microscopic studies performed after surgical removal of the lesion demonstrated high amount of mesenchymal fusiform cells combined with vascular structures  axial and coronal reformations of contrast  enhanced ct on portal  a  b  and  min after injection delayed acquisitions  c  d   pararectal solid and well  defined mass with moderated peripheral enhancement adjacent to the rectum and left seminal vesicle was detected  mesenchymal neoplasm composed of polygonal cells with clear cytoplasm and round normochromatic nuclei with abundant vessels and occasional scattered inflammatory cells was observed  mixed within the neoplasm cells  there was a chronic inflammatory infiltrate of lymphoid cells  pathology  metastatic imt of the appendix  a year  old man with antecedents of left hemicolectomy due to colon cancer  a axial reformation of contrast  enhanced ct on portal phase acquired seven years after partial colectomy  an appendicular  irregular  solid mass with slight enhancement was identified  yellow arrow   due to its radiological features  b axial reformation of contrast  enhanced ct on portal phase performed  months later showed a significant growth of the appendicular mass with extension and infiltration of the adjacent small bowel loops  axial reformation of contrast  enhanced ct on arterial  a   portal  b  and  min after injection delayed  c  acquisitions  a well  defined  heterogeneous nodule with moderate enhancement on portal phase arising from the gastric fundus was found  yellow arrows   d microscopic studies performed after surgical removal of the lesion demonstrated high amount of mesenchymal fusiform cells combined with vascular structures  the lesion showed well  defined contours and an expansive growth pattern pathology  pararectal imt  axial and coronal reformations of contrast  enhanced ct on portal  a  b  and  min after injection delayed acquisitions  c  d   pararectal solid and well  defined mass with moderated peripheral enhancement adjacent to the rectum and left seminal vesicle was detected  mesenchymal neoplasm composed of polygonal cells with clear cytoplasm and round normochromatic nuclei with abundant vessels and occasional scattered inflammatory cells was observed  mixed within the neoplasm cells  there was a chronic inflammatory infiltrate of lymphoid cells  there were no foci of necrosis or mitotic figures pathology  metastatic imt of the appendix  a year  old man with antecedents of left hemicolectomy due to colon cancer  a axial reformation of contrast  enhanced ct on portal phase acquired seven years after partial colectomy  an appendicular  irregular  solid mass with slight enhancement was identified  yellow arrow   due to its radiological features  b axial reformation of contrast  enhanced ct on portal phase performed  months later showed a significant growth of the appendicular mass with extension and infiltration of the adjacent small bowel loops  c distant lymphadenopathy and liver metastases were present the differential diagnosis of these tumours includes gist  inflammatory fibroid polyp  smooth muscle neoplasm  peripheral nerve sheath tumour  solitary fibrous tumour  fibromatosis  the follicular dendritic cell sarcoma  lymphoma and adenocarcinomas  in most of these cases imts of the gastrointestinal tract present a higher recurrence rate than in other locations of the body  other treatments  such as steroids  non  steroidal anti  inflammatory drugs and thalidomide  have been used in these tumours  radiotherapy may have a place as adjunctive therapy in local recurrence and incomplete surgical removal  imts of the appendix are extremely rare  with few cases reported in the literature         chronic infections  antecedents of trauma or surgery have been proposed as possible aetiological factors  extremely rare cases of imts of the appendix showing local aggressiveness and metastases have been reported  fig  many different aetiological factors have been proposed for mesenteric imts       such as chronic infections  autoimmune diseases and trauma  on contrast  enhanced ct they usually appear as solid masses with heterogeneous enhancement pattern and ill or well  defined margins  they are often associated with a general inflammatory response manifested with fever  weight loss and other symptoms that are related to the compressive effect of the tumour  such as abdominal pain and vomiting  rapidly growing tumours or symptomatic ones needing treatment may follow from small tumours which are not encroaching any nearby structures  different treatments have been proposed  non  steroidal anti  inflammatories  anti  oestrogens  chemotherapeutic agents and surgery  surgery remains as a useful modality although quite a high local recurrence rate has been described for tumours         the symptoms that most often appear include pain  haematuria  dysuria or urinary tract obstruction  the radiological features do not help to distinguish them from malignant neoplasms  such as rabdomyosarcomas  fig   fine  needle biopsy may fail to yield a sufficient volume of tumour tissue to achieve diagnosisfig  a axial and b coronal reformations of contrast  enhanced ct where an ill  defined and heterogeneous mass encompassing iliopsoas muscle and external iliac vessels  mimicking malignant sarcomatous neoplasm was observed  proliferation of fibroblast with a diffuse infiltration of lymphocytes  eosinophils and macrophages were found  the mass was well  defined and no signs of invasiveness were found pathology  imt of the iliopsoas muscle  a axial and b coronal reformations of contrast  enhanced ct where an ill  defined and heterogeneous mass encompassing iliopsoas muscle and external iliac vessels  mimicking malignant sarcomatous neoplasm was observed  proliferation of fibroblast with a diffuse infiltration of lymphocytes  eosinophils and macrophages were found  no mitotic figures or cellular atypia were present  imts have been reported in various sites in the head and neck such as bucal space  maxillary sinus  nasal cavity  parotid gland  nasopharynx and larynx  fig  pathology  imt in the left supraglottic space  arrow  with paraglottic space involvement indicated by replacement of the paraglottic fat with soft tissue  a axial reformation of contrast  enhanced neck ct  a solid  well  defined  normal tissue was present in the peripheral zone    pathology  imt in the left supraglottic space  arrow  with paraglottic space involvement indicated by replacement of the paraglottic fat with soft tissue  a axial reformation of contrast  enhanced neck ct  a solid  well  defined  little  enhancing nodule in the supraglottic larynx was observed  yellow arrow   b microscopic study obtained after surgical removal of the lesion  the sample showed proliferation of fusiform cells mixed with macrophages and giant multinucleated cells  normal tissue was present in the peripheral zone    however  the orbit is the most common location of imts  they account for approximately   of orbital masses  being the third most common cause of orbital masses  their aetiology is unknown  but they have been associated with inmunological disorders  different infections and history of trauma  recently  they have been strongly associated with the iggrelated sclerosing systemic inflammatory disease and with chronic infection by epstein  barr virus  the clinical presentation and symptoms may vary with the location  extension and aggressiveness of the tumour  patients with imts of the orbit can present local symptoms as pain  redness  oedema  proptosis  ptosis  oculomotor deficits  diplopia  lid swelling or mass  effect and systemic symptoms due to the general inflammatory disorders they are frequently associated with  mri is superior to ct in the evaluation of inflammation of the nerves and muscles  they usually appear as solid and heterogeneous enhancing masses  which can affect retro bulbar fat  cause bone destruction  and present intracranial extension  when they show low intensity on tweighted images prominent  fibrotic composition is present  fig  a  b axial and coronal mr reformations of the same patient on tweighted sequences showed a well  delimited  hyperintense intraorbital mass  in the intraconal compartment  in the medial aspect  c  d tweighted images demonstrated a hypointense  homogeneous  well  delimited mass with peripheral enhancement after gadolinium injection  f microscopic sample obtained after surgery demonstrated an epithelial tumour composed of fusiform cells mixed with an extensive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  in the peripheral of the sample a  b axial and coronal mr reformations of the same patient on tweighted sequences showed a well  delimited  hyperintense intraorbital mass  in the intraconal compartment  in the medial aspect  c  d tweighted images demonstrated a hypointense  homogeneous  well  delimited mass with peripheral enhancement after gadolinium injection  f microscopic sample obtained after surgery demonstrated an epithelial tumour composed of fusiform cells mixed with an extensive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  in the peripheral of the sample striated muscle cells were also observed pathologically  imts of the head and neck consist of many fibroblastic cells  fibroblastic cells and inflammatory cells  which include plasma cells and eosinophils  the differential diagnosis includes granulomatous diseases  primary infection  sarcoid  sjgren disease  primary and secondary tumours of the orbit  lymphoma and connective tissue diseases  the administration of corticosteroids an inmunosupresors usually leads to a decrease of size of the mass but also radiotherapy or surgery may be indicated   they account for around the   of benign pulmonary masses in children  representing only about   of all tumours of the lung in general population  as in other organs  they are also associated with immunological disorders and chronic infections  sometimes  imts are also related with antecedents of lung surgery and they can arise in surgical lung scars  common clinical characteristics include unspecific respiratory symptoms  such as cough  chest pain  dyspnea  haemoptysis and unspecific inflammatory symptoms as fever  malaise and weight loss  they usually appear as solitary  well  circumscribed peripheral lung masses  with predominance for the lower lobes  fig  on ct with intravenous contrast  they present a variable heterogeneous or homogeneous degree of enhancement pattern       it has been described an aggressive behaviour with invasion of the adjacent structures  on mri because of their similar radiological appearance to malignant lung masses  biopsy is necessary to obtain the diagnosisfig  a year  old man with a cough and haemoptysisa axial reformation of non  contrast ct with mediastinum window  two solid  ill  defined nodules in right lower lobe are observed  yellow arrows   b axial reformations with lung window of the same patient where the lung nodules are shown  white arrows   c partial lobectomy of the right lower lobe specimen  note the presence of the nodules     d microscopic study of the resected lung sample where fusiform cells with an associated inflammatory infiltrate of lymphocytes  plasmatic cells and histiocytes was found pathology  imt of the lung  a year  old man with a cough and haemoptysisa axial reformation of non  contrast ct with mediastinum window  two solid  ill  defined nodules in right lower lobe are observed  yellow arrows   note the air brocogram observed in the biggest mass  b axial reformations with lung window of the same patient where the lung nodules are shown  white arrows   d microscopic study of the resected lung sample where fusiform cells with an associated inflammatory infiltrate of lymphocytes  plasmatic cells and histiocytes was found the treatment of choice of lung imts is surgery to exclude malignancy and to achieve the cure  although spontaneous regression may occur  local expansion may cause significant morbidity and occasional death  the prognosis after complete surgical resection is excellent  with a low rate of recurrence  in these cases  alternative treatments  radiotherapy  corticoids or chemotherapy  to surgery could be also used  the most common presentation of epididymal  fig    and paratesticular imts is a lump in the scrotumfig  pathology  imt of the epididymus  a year  old man with a lump in the scrotum  a a hypoechoic a diffuse infiltrate composed of fibroblastic and fusiform cells mixed with inflammatory cells  mainly lymphocytes and macrophages  was observed pathology  imt of the epididymus  a year  old man with a lump in the scrotum  a a hypoechoic  well  delimited epididymal lesion in the right scrotum was detected  a diffuse infiltrate composed of fibroblastic and fusiform cells mixed with inflammatory cells  mainly lymphocytes and macrophages  was observed in the evaluation of scrotal masses  us is the most accurate imaging technique  helping to distinguish intratesticular from extratesticular lesions and solid from cystic lesions  on ct  they appear as well  defined  hypodense masses with little enhancement after intravenous contrast injection  in some cases  a year  old man with no relevant medical history came to our hospital with a lump in the scrotum  a us showed an isoechoic  solid  paratesticular nodule with fine calcifications  yellow arrow   b axial reformation of contrast  enhanced ct on portal phase demonstrated a well  delimited  heterogeneous  little  enhancing mass in the right scrotum  d microscopic study of the lesion demonstrated a well  delimited lesion constituted of fibroblasts  lymphocytes and plasmatic cells with associated lymphoid follicle  a year  old man with no relevant medical history came to our hospital with a lump in the scrotum  a us showed an isoechoic  solid  paratesticular nodule with fine calcifications  yellow arrow   b axial reformation of contrast  enhanced ct on portal phase demonstrated a well  delimited  heterogeneous  little  enhancing mass in the right scrotum  d microscopic study of the lesion demonstrated a well  delimited lesion constituted of fibroblasts  lymphocytes and plasmatic cells with associated lymphoid follicle  there was no evidence of adjacent testicle invasion surgical resection without orchiectomy is the ideal treatment  but in some cases the testicle must be removed due to the size of the mass or when the mass is attached to the testis  imts of the penis are extremely rare  to our knowledge  no imts at this location have been reported  we have recorded only one case in our institution  fig     on ct  this tumour appeared as a well  defined  little  enhancing and non  aggressive mass surrounding the penile urethra and the corpora cavernosa with no evidence of invasion of themfig  a year  old man with a mass in the penis came to our hospital  he related more than  years of slow growth of the mass with no suspicious associated symptoms  c  d axial and coronal reformations of enhanced  ct after the injection of intravenous contrast on portal phase  macroscopic view showed an heterogeneous aspect due to the mixture of tissues  solid tumour  lipoid component and areas of haemorrhage   f microscopic study confirmed a mesenchymal tumour composed of fusiform cells  some of them multinucleated  with round or elongate nucleus  a year  old man with a mass in the penis came to our hospital  he related more than  years of slow growth of the mass with no suspicious associated symptoms  c  d axial and coronal reformations of enhanced  ct after the injection of intravenous contrast on portal phase  macroscopic view showed an heterogeneous aspect due to the mixture of tissues  solid tumour  lipoid component and areas of haemorrhage   f microscopic study confirmed a mesenchymal tumour composed of fusiform cells  some of them multinucleated  with round or elongate nucleus  no mitotic figures were observed surgical treatment was performed due to the size and non  conclusive radiological features of the tumour  hepatic imts constitute an extremely rare group of hepatic tumours  accounting only for the   of all the hepatic lesions  they have been associated with autoimmune diseases  immunological disorders and viral infections  such as igg disease  sclerosing cholangitis  crohns disease  hepatic viral infections and epstein  barr virus  on contrast  enhanced ct  they usually show heterogeneous or peripheral enhancement during the arterial phase  despite this  some of them present homogeneous enhancement during the arterial phase and washout during the delayed phase  which can lead to misdiagnosing them as hepatocarcinomas  hccs   fig     on mri  these lesions usually are t hypointense and t hyperintense with heterogeneous enhancement  fig    hepatobiliary contrast  gadoxetate disodium  primovist  bayer healthcare  leverkusen  germany  could be of help to differentiate hcc from imts of the liver  on the hepatobiliary phase  high signal intensity in the centre of the lesion has been described  suggesting a benign lesionfig  well  defined  hypointense lesion on tweighted images  a  which presented weak enhancement in the arterial phase  b  and wash  out on portal and equilibrium phases  c  d  located in hepatic left lobe  f the lesion demostrated an expansive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  a abdominal ultrasound where a slightly hypoechoic  solid  heterogeneous mass arising in the left hepatic lobe was observed  axial tweighted unenhanced  b  and contrast  enhanced mr images  c  d  showed an ill  defined with heterogeneous enhancement mass located in lateral hepatic segments  mesenchymal tumour composed of fusiform cells  lymphocytes and lipid  filled macrophages can be observed  normal hepatocytes with large and small fatty droplet change are also present pathology  imt of the liver  well  defined  hypointense lesion on tweighted images  a  which presented weak enhancement in the arterial phase  b  and wash  out on portal and equilibrium phases  c  d  located in hepatic left lobe  e macroscopic view of partial hepatectomy obtained after surgical excision of the lesion  a solid  well  defined and yellowing appearance mass is showed  f the lesion demostrated an expansive chronic inflammatory infiltrate of plasmatic cells  lymphocytes and macrophages  a abdominal ultrasound where a slightly hypoechoic  solid  heterogeneous mass arising in the left hepatic lobe was observed  axial tweighted unenhanced  b  and contrast  enhanced mr images  c  d  showed an ill  defined with heterogeneous enhancement mass located in lateral hepatic segments  mesenchymal tumour composed of fusiform cells  lymphocytes and lipid  filled macrophages can be observed  normal hepatocytes with large and small fatty droplet change are also present surgery is the most accepted treatment  if there is a biopsy  proven diagnosis of imts that excludes malignancy  medical treatment with no steroidal anti  inflammatory drugs in patients with peripheral hepatic imts could be prescribed  patients with spleen imts frequently associate unspecific symptoms as left upper quadrant or epigastric pain  splenomegaly is common and some patients can present fever  anaemia and signs of hypersplenism  different radiological imaging modalities have been proposed for their evaluation  on all of them  they are depicted as low  density masses in both non  enhanced and enhanced acquisitions  fig  their typical enhancement pattern after gadolinium injection is as a low  intensity lesion in the early phase  which changes into a high  intensity one in the delayed phase fig  pathology  imt of the spleen  a axial reformation of contrast  enhanced ct on portal phase  prominent cellularity composed of an irregular  weakly basophil set of mioepithelial  fusiform cells  numerous inflammatory cells  plasmatic  lymphocytes and few neutrophils and eosinophils  with isolated macrophages were also found  no atypia or mitotic figures were present pathology  imt of the spleen  a axial reformation of contrast  enhanced ct on portal phase  prominent cellularity composed of an irregular  weakly basophil set of mioepithelial  fusiform cells  numerous inflammatory cells  plasmatic  lymphocytes and few neutrophils and eosinophils  with isolated macrophages were also found  no atypia or mitotic figures were present the differential diagnosis includes splenic haemangioma and other primary splenic neoplasms such as lymphoma or splenic angiosarcoma  when a suspicious spleen mass is found  the treatment of choice is splenectomy due to the high risk of bleeding associated to splenic biopsy       they can arise anywhere  the stomach and small intestine being the most common locations       clinical symptoms may vary from none to abdominal pain  intestinal obstruction  dysphagia or anaemia  depending on the location and size of the mass  contrast  enhanced ct is the imaging technique  which give us more information about of imts of the gastrointestinal tract  they are usually found as hypodense  heterogeneous and well  delimited masses  figs  axial reformation of contrast  enhanced ct on arterial  a   portal  b  and  min after injection delayed  c  acquisitions  a well  defined  heterogeneous nodule with moderate enhancement on portal phase arising from the gastric fundus was found  yellow arrows   d microscopic studies performed after surgical removal of the lesion demonstrated high amount of mesenchymal fusiform cells combined with vascular structures  axial and coronal reformations of contrast  enhanced ct on portal  a  b  and  min after injection delayed acquisitions  c  d   pararectal solid and well  defined mass with moderated peripheral enhancement adjacent to the rectum and left seminal vesicle was detected  e macroscopic view of the tumour after surgical resection  f histological sample obtained after biopsy  mesenchymal neoplasm composed of polygonal cells with clear cytoplasm and round normochromatic nuclei with abundant vessels and occasional scattered inflammatory cells was observed  mixed within the neoplasm cells  there was a chronic inflammatory infiltrate of lymphoid cells  pathology  metastatic imt of the appendix  a year  old man with antecedents of left hemicolectomy due to colon cancer  a axial reformation of contrast  enhanced ct on portal phase acquired seven years after partial colectomy  an appendicular  irregular  solid mass with slight enhancement was identified  yellow arrow   due to its radiological features  b axial reformation of contrast  enhanced ct on portal phase performed  months later showed a significant growth of the appendicular mass with extension and infiltration of the adjacent small bowel loops  axial reformation of contrast  enhanced ct on arterial  a   portal  b  and  min after injection delayed  c  acquisitions  a well  defined  heterogeneous nodule with moderate enhancement on portal phase arising from the gastric fundus was found  yellow arrows   d microscopic studies performed after surgical removal of the lesion demonstrated high amount of mesenchymal fusiform cells combined with vascular structures  the lesion showed well  defined contours and an expansive growth pattern pathology  pararectal imt  axial and coronal reformations of contrast  enhanced ct on portal  a  b  and  min after injection delayed acquisitions  c  d   pararectal solid and well  defined mass with moderated peripheral enhancement adjacent to the rectum and left seminal vesicle was detected  mesenchymal neoplasm composed of polygonal cells with clear cytoplasm and round normochromatic nuclei with abundant vessels and occasional scattered inflammatory cells was observed  mixed within the neoplasm cells  there was a chronic inflammatory infiltrate of lymphoid cells  there were no foci of necrosis or mitotic figures pathology  metastatic imt of the appendix  a year  old man with antecedents of left hemicolectomy due to colon cancer  a axial reformation of contrast  enhanced ct on portal phase acquired seven years after partial colectomy  an appendicular  irregular  solid mass with slight enhancement was identified  yellow arrow   due to its radiological features  b axial reformation of contrast  enhanced ct on portal phase performed  months later showed a significant growth of the appendicular mass with extension and infiltration of the adjacent small bowel loops  c distant lymphadenopathy and liver metastases were present the differential diagnosis of these tumours includes gist  inflammatory fibroid polyp  smooth muscle neoplasm  peripheral nerve sheath tumour  solitary fibrous tumour  fibromatosis  the follicular dendritic cell sarcoma  lymphoma and adenocarcinomas  in most of these cases imaging features are not enough to diagnose imts and biopsy or surgery are needed  imts of the gastrointestinal tract present a higher recurrence rate than in other locations of the body  other treatments  such as steroids  non  steroidal anti  inflammatory drugs and thalidomide  have been used in these tumours  radiotherapy may have a place as adjunctive therapy in local recurrence and incomplete surgical removal  imts of the appendix are extremely rare  with few cases reported in the literature         chronic infections  antecedents of trauma or surgery have been proposed as possible aetiological factors  extremely rare cases of imts of the appendix showing local aggressiveness and metastases have been reported  fig  many different aetiological factors have been proposed for mesenteric imts       such as chronic infections  autoimmune diseases and trauma  on contrast  enhanced ct they usually appear as solid masses with heterogeneous enhancement pattern and ill or well  defined margins  they are often associated with a general inflammatory response manifested with fever  weight loss and other symptoms that are related to the compressive effect of the tumour  such as abdominal pain and vomiting  rapidly growing tumours or symptomatic ones needing treatment may follow from small tumours which are not encroaching any nearby structures  different treatments have been proposed  non  steroidal anti  inflammatories  anti  oestrogens  chemotherapeutic agents and surgery  surgery remains as a useful modality although quite a high local recurrence rate has been described for tumours         the symptoms that most often appear include pain  haematuria  dysuria or urinary tract obstruction  the radiological features do not help to distinguish them from malignant neoplasms  such as rabdomyosarcomas  fig   fine  needle biopsy may fail to yield a sufficient volume of tumour tissue to achieve diagnosisfig  a axial and b coronal reformations of contrast  enhanced ct where an ill  defined and heterogeneous mass encompassing iliopsoas muscle and external iliac vessels  mimicking malignant sarcomatous neoplasm was observed  proliferation of fibroblast with a diffuse infiltration of lymphocytes  eosinophils and macrophages were found  no mitotic figures or cellular the mass was well  defined and no signs of invasiveness were found pathology  imt of the iliopsoas muscle  a axial and b coronal reformations of contrast  enhanced ct where an ill  defined and heterogeneous mass encompassing iliopsoas muscle and external iliac vessels  mimicking malignant sarcomatous neoplasm was observed  proliferation of fibroblast with a diffuse infiltration of lymphocytes  eosinophils and macrophages were found  no mitotic figures or cellular although imts in some organs are not uncommon  they are not usually included in the differential diagnosis of nodules and masses  consequently  this is an underdiagnosed entity and only after an histological exam could definitive diagnosis be achieved  it is fundamental for radiologists to know about the existence of imts  as they frequently have better treatment options and prognosis than the malignant neoplasms they are confused with",

"""

summary = generateSummary(new_article_text)
print("Generated Summary:", summary)


Generated Summary: inflammatory myofibroblastic tumour imts constitute rare group neoplasm compose mixture spindle shape myofibroblast fibroblast variable inflammatory cell eosinophil plasma cell lymphocyte different term refer tumour plasma cell granuloma inflammatory myofibrohistiocytic proliferation fibroxanthoma xanthogranuloma frequently affect organ lung orbit describe clinical presentation imts depend organ arise frequently associate general inflammatory disease igg disease recently associate aggressive


SyntaxError: invalid syntax (4075987369.py, line 7)

C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Generated Summary:
example article want summarize contain multiple sentence special character like comma period text include number web link comma period text comma period text include number web link comma period text comma period text comma period text comma period text comma period text comma period text comma period text comma period text comma period text comma period text comma period text comma period text comma period


2024-06-24 03:11:02.539 
  command:

    streamlit run C:\ProgramData\anaconda3\envs\pubMed\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-24 03:11:02.540 Session state does not function when running a script without `streamlit run`


In [2]:
import yaml

# Load the environment.yml file
with open('environment.yml', 'r') as file:
    env = yaml.safe_load(file)

# Extract the dependencies
dependencies = env['dependencies']

# Initialize a list to hold the requirements
requirements = []

# Process each dependency
for dep in dependencies:
    if isinstance(dep, str):
        # Simple string dependencies can be added directly
        requirements.append(dep)
    elif isinstance(dep, dict) and 'pip' in dep:
        # Pip dependencies are listed inside a dict
        requirements.extend(dep['pip'])

# Write the requirements to a requirements.txt file
with open('requirements.txt', 'w') as file:
    for req in requirements:
        file.write(f"{req}\n")


ReaderError: unacceptable character #x0000: special characters are not allowed
  in "environment.yml", position 3